# Tests unitaires de l'API
## Scénario de test unitaire 
Vu qu'il s'agit d'un modèle de classification binaire, le test unitaire consistera à :
- faire prédire la variable cible par l'API du modèle 
- fournir au modèle les valeurs des attributs via un formulaire

Le micro-framework Web léger [_Flask_](https://flask.palletsprojects.com/en/3.0.x/quickstart/) prend en charge la gestion de requêtes et réponses REST

In [77]:
ip_server = '13.92.86.145'
port_api_server, port_web_server = '5678', '6543'
url_api_server = ip_server + ':' + port_api_server + '/invocations'

## Tests avec _pytest_

#### Installation de _pytest_

In [15]:
! pip install pytest

Found existing installation: pytest 8.2.1
Uninstalling pytest-8.2.1:
  Would remove:
    /home/azureuser/environments_folder/my_env/bin/py.test
    /home/azureuser/environments_folder/my_env/bin/pytest
    /home/azureuser/environments_folder/my_env/lib/python3.10/site-packages/_pytest/*
    /home/azureuser/environments_folder/my_env/lib/python3.10/site-packages/py.py
    /home/azureuser/environments_folder/my_env/lib/python3.10/site-packages/pytest-8.2.1.dist-info/*
    /home/azureuser/environments_folder/my_env/lib/python3.10/site-packages/pytest/*
Proceed (Y/n)? ^C
ERROR: Operation cancelled by user


#### Fonctions de test

In [4]:
!cat ../test_api/test_2.py

import requests
import json

def predict(dict_features):
    # Send input data to prediction API
    req_post = requests.post(   url     = 'http://13.92.86.145:5678/invocations',
                                headers = {'Content-Type': 'application/json'},
                                data    = json.dumps({'inputs': dict_features}) )
    # Get predicted value from API
    dict_predicted = json.loads(req_post.text)
    return dict_predicted

def test_prediction():
    for idx, (in_1, in_2, out_i) in enumerate([
                        (.2, .2, 1),
                        (.3, .3, 1),
                        (.4, .4, 0)   ]) :
        print('_____________Unit Test N°', idx + 1, '_______________')
        dict_features_i  = {'umap_x':[str(in_1)],'umap_y':[str(in_2)]}
        dict_predicted_i = predict(dict_features_i)
        dict_expected_i  = {'predictions':[out_i]}
        print('input features   :', dict_features_i)
        print('output predicted:', dict_predicted_i)
        pri

#### Exécution et résultats des tests

In [17]:
!python -m pytest --import-mode=append -rA ../test_api/

============================= test session starts ==============================
platform linux -- Python 3.10.12, pytest-8.2.1, pluggy-1.5.0
rootdir: /home/azureuser/project_7
plugins: anyio-4.3.0
collected 2 items                                                              

../test_api/test_1.py .                                                  [ 50%]
../test_api/test_2.py .                                                  [100%]

==================================== PASSES ====================================
_______________________________ test_connection ________________________________
----------------------------- Captured stdout call -----------------------------
__________Connection to API Server_____________
OK : Connected to API Server
_______________________________ test_prediction ________________________________
----------------------------- Captured stdout call -----------------------------
_____________Unit Test N° 1 _______________
input features   : {'umap_x': ['0.

**Observation** : les résultats des tests unitaires sont positifs

## Tests via _curl_

In [ ]:
import subprocess

In [81]:
def get_single_prediction(str_attributes, str_expected) :
    shell_command = 'curl \
        -d \'{"dataframe_split": { "columns": ["umap_x","umap_y"], "data": [[' + str_attributes + ']]}}\' \
        -H \'Content-Type: application/json\' -X POST ' + url_api_server + ' --no-progress-meter'
    print(subprocess.getoutput(shell_command), 'expected :', str_expected, 'for attributes:', str_attributes)
get_single_prediction('-0.749079, 1.281123', '0')

{"predictions": [0]} expected : 0 for attributes: -0.749079, 1.281123


## Tests via interface Web d'API

### Accès en ligne

In [45]:
print('Website URL =' , 'http://' + ip_server + ':' + port_web_server)

Website URL = http://13.92.86.145:6543


### Arborescence de fichiers

In [24]:
! find ../website | sed -e "s/[^\/]*\//  |/g" -e "s/|\([^ ]\)/└── \1/"

  └── website
  |  └── .ipynb_checkpoints
  |  |  └── Jean_Vallée_5_notebook_test_API_042024-checkpoint.ipynb
  |  └── web_router.py
  |  └── templates
  |  |  └── result.html
  |  |  └── index.html
  |  |  └── form.html
  |  └── static
  |  |  └── css
  |  |  |  └── style.css
  |  └── Jean_Vallée_5_notebook_test_API_042024.ipynb


### Fichiers _HTML_

In [19]:
dir_html = './templates/'

#### Index  
Fichier d'accueil

In [21]:
file_html_index = dir_html + 'index.html'
!cat $file_html_index


<!DOCTYPE html>
<html lang="en">
    <head>
        <meta charset="UTF-8">
        <link rel="stylesheet" href="{{ url_for('static', filename= 'css/style.css') }}">
        <title>ML model API</title>
    </head>
    <body>
       <h1>API du modèle de classification binaire</h1>      
       <h2>Page d'accueil</h2>    
       <a href="/form/">Formulaire</a>
    </body>
</html>



In [33]:
import IPython
IPython.display.IFrame(file_html_index, width=700, height=350) 

#### Formulaire  
Formulaire de saisie des atttributs

In [37]:
file_html_form = dir_html + 'form.html'
!cat $file_html_form


<!DOCTYPE html>
<html lang="en">
    <head>
        <meta charset="UTF-8">
        <link rel="stylesheet" href="{{ url_for('static', filename= 'css/style.css') }}">
        <title>ML model API</title>
    </head>
    <body>
        <h1>API du modèle de classification binaire</h1>       
        <h2>Formulaire de saisie d'attributs</h2>       
        
        <form method="post" action="{{ url_for('result') }}">
            <div>
                <label for="feat1">Feature 1 :</label>
                <input name="umap_x" id="feat1" type="number" step="any" value="0.3">
            </div>
            <div>
                <label for="feat2">Feature 2 :</label>
                <input name="umap_y" id="feat2" type="number" step="any" value="0.3">
            </div>
            <div>
                <button>Predict</button>
            </div>
        </form>
      
    </body>
</html>



In [38]:
IPython.display.IFrame(file_html_form, width=700, height=350) 

#### Résultat
Page de résultat de prédiction par l'API du modèle

In [39]:
file_html_result = dir_html + 'result.html'
!cat $file_html_result


<!DOCTYPE html>
<html lang="en">
    <head>
        <meta charset="UTF-8">
        <link rel="stylesheet" href="{{ url_for('static', filename= 'css/style.css') }}">
        <title>ML model API</title>
    </head>
    <body>
       <h1>API du modèle de classification binaire</h1>      
       <h2>Page de prédiction du modèle</h2> 
       <p>umap_x = {{features['umap_x']}}</p>
       <p>umap_y = {{features['umap_y']}}</p>
       <p>valeur prédite = {{target_value['predictions']}}</p>
       <a href="/">Retour à l'accueil</a>
       <a href="/form/">Formulaire</a>
    </body>
</html>



In [40]:
IPython.display.IFrame(file_html_result, width=700, height=350) 

#### Style

In [10]:
dir_style = './static/css/'

In [11]:
file_style = dir_style + 'style.css'
!cat $file_style


h1 {
    border: 2px #eee solid;
    color: brown;
    text-align: center;
    padding: 10px;
}



### Fichiers _Python_

#### Routeur Web  
Un fichier en Pyhton couvre la gestion REST

In [23]:
file_api = 'web_router.py'
!cat $file_api


from flask import Flask, render_template, request
import requests
import json

app = Flask(__name__) # get work dirs

@app.route('/')                                           # index
def index():
    return render_template('index.html') # HTML as string

@app.route('/form/')                                      # form
def form():
    return render_template('form.html')

@app.route('/result/', methods=['POST'])                  # result predicted by API
def result():    
    dict_features = request.form.to_dict(flat=False) #{umap_x:[.3],umap_y:[.3]}
    req_post = requests.post(   url     = 'http://localhost:5678/invocations', 
                                headers = {'Content-Type': 'application/json'}, 
                                data    = json.dumps({'inputs': dict_features}) )
    dict_prediction = json.loads(req_post.text)     #{predictions:[1]}
    return render_template('result.html', features=dict_features, target_value=dict_prediction)

@app.route('/deploy/')          

## Serveur _Web_

### Lancement

In [26]:
shell_command = 'flask --app $file_api run -h 0.0.0.0 -p 6543 --debug > ./flask_app.log 2>&1 &'
get_ipython().system_raw(shell_command) # run model API in background

### Vérifications
#### Log

In [27]:
!tail ./flask_app.log

 * Serving Flask app 'web_router.py'
 * Debug mode: on
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:6543
 * Running on http://10.0.0.4:6543
Press CTRL+C to quit
 * Restarting with stat
 * Debugger is active!
 * Debugger PIN: 134-700-135


#### Processus
Liste des processus

In [28]:
li_ps = !ps aux | grep "flask" | grep -v "grep" | awk '{print $2}' 
!ps aux | grep "flask" | grep -v "grep" 

azureus+  131413  1.0  0.9  37648 31896 ?        S    21:31   0:00 /home/azureuser/environments_folder/my_env/bin/python3 /home/azureuser/environments_folder/my_env/bin/flask --app web_router.py run -h 0.0.0.0 -p 6543 --debug
azureus+  131414  1.5  0.9 111380 31768 ?        Sl   21:31   0:00 /home/azureuser/environments_folder/my_env/bin/python3 /home/azureuser/environments_folder/my_env/bin/flask --app web_router.py run -h 0.0.0.0 -p 6543 --debug


Arrêt des processus

In [29]:
for ps_i in li_ps : 
    #!kill -9 $ps_i

SyntaxError: incomplete input (2464387668.py, line 2)